In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
from scipy.constants import R
from openap import prop
from pprint import pprint

In [3]:
df_challenge = pd.read_csv('../data/challenge_set.csv')
df_trajectory = pd.read_parquet('../data/2022-01-01.parquet')

## Data prep

In [4]:
columns_with_nan = df_trajectory.columns[df_trajectory.isna().any()].tolist()
columns_with_nan

['groundspeed', 'track', 'vertical_rate']

In [5]:
df_trajectory['track'] = df_trajectory['track'].interpolate()
df_trajectory['vertical_rate'] = df_trajectory['vertical_rate'].interpolate()

In [6]:
df_trajectory['groundspeed'] = df_trajectory['groundspeed'].interpolate()
df_trajectory

,flight_id,timestamp,latitude,longitude,altitude,groundspeed,track,vertical_rate,icao24,u_component_of_wind,v_component_of_wind,temperature,specific_humidity
0,248763775,2022-01-01 18:14:38+00:00,30.898407,31.240270,38000.0,429.0,350.888643,-64.0,248763775,8.754636,-13.070078,221.819618,0.000008
1,248763775,2022-01-01 18:14:39+00:00,30.909615,31.238251,38000.0,429.0,351.020434,128.0,248763775,8.696749,-13.106524,221.794782,0.000008
2,248763775,2022-01-01 18:14:40+00:00,30.909615,31.238251,38000.0,429.0,351.020434,128.0,248763775,8.697087,-13.107184,221.794700,0.000008
3,248763775,2022-01-01 18:14:41+00:00,30.909615,31.238251,38000.0,429.0,351.020434,128.0,248763775,8.697424,-13.107845,221.794618,0.000008
4,248763775,2022-01-01 18:14:42+00:00,30.909615,31.238251,38000.0,429.0,351.020434,128.0,248763775,8.697762,-13.108505,221.794536,0.000008
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11862458,248772010,2022-01-02 01:59:55+00:00,44.065750,-58.601074,40975.0,543.0,76.040526,-64.0,248772010,49.665934,-31.158741,210.208665,0.000013
11862459,248772010,2022-01-02 01:59:56+00:00,44.065750,-58.601074,40975.0,543.0,76.040526,-64.0,248772010,49.665919,-31.158539,210.208654,0.000013
11862460,248772010,2022-01-02 01:59:57+00:00,44.065750,-58.601074,40975.0,543.0,76.040526,-64.0,248772010,49.665905,-31.158336,210.208643,0.000013
11862461,248772010,2022-01-02 01:59:58+00:00,44.065750,-58.601074,40975.0,543.0,76.040526,-64.0,248772010,49.665890,-31.158134,210.208632,0.000013


In [7]:
df_combined = pd.merge(df_challenge, df_trajectory, on='flight_id', how='inner')
df_combined

,flight_id,date,callsign,adep,name_adep,country_code_adep,ades,name_ades,country_code_ades,actual_offblock_time,...,longitude,altitude,groundspeed,track,vertical_rate,icao24,u_component_of_wind,v_component_of_wind,temperature,specific_humidity
0,248763780,2022-01-01,3840d84f25d3f5fcc0a1be3076bb4039,EGLL,London Heathrow,GB,EICK,Cork,IE,2022-01-01T13:46:00Z,...,-0.443802,25.0,230.846154,199.229241,152.615385,248763780,2.329204,6.897236,287.955749,0.008058
1,248763780,2022-01-01,3840d84f25d3f5fcc0a1be3076bb4039,EGLL,London Heathrow,GB,EICK,Cork,IE,2022-01-01T13:46:00Z,...,-0.443802,25.0,224.692308,205.065984,241.230769,248763780,2.329413,6.897262,287.955597,0.008058
2,248763780,2022-01-01,3840d84f25d3f5fcc0a1be3076bb4039,EGLL,London Heathrow,GB,EICK,Cork,IE,2022-01-01T13:46:00Z,...,-0.443802,25.0,218.538462,210.902727,329.846154,248763780,2.329622,6.897288,287.955446,0.008058
3,248763780,2022-01-01,3840d84f25d3f5fcc0a1be3076bb4039,EGLL,London Heathrow,GB,EICK,Cork,IE,2022-01-01T13:46:00Z,...,-0.443802,25.0,212.384615,216.739470,418.461538,248763780,2.329831,6.897314,287.955294,0.008058
4,248763780,2022-01-01,3840d84f25d3f5fcc0a1be3076bb4039,EGLL,London Heathrow,GB,EICK,Cork,IE,2022-01-01T13:46:00Z,...,-0.443802,25.0,206.230769,222.576213,507.076923,248763780,2.330040,6.897341,287.955142,0.008058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4128685,248772010,2022-01-02,3008152ce54bc95bc78b003206dc45a7,KPHL,Philadelphia,US,LSZH,Zurich,CH,2022-01-02T00:01:00Z,...,-58.601074,40975.0,543.000000,76.040526,-64.000000,248772010,49.665934,-31.158741,210.208665,0.000013
4128686,248772010,2022-01-02,3008152ce54bc95bc78b003206dc45a7,KPHL,Philadelphia,US,LSZH,Zurich,CH,2022-01-02T00:01:00Z,...,-58.601074,40975.0,543.000000,76.040526,-64.000000,248772010,49.665919,-31.158539,210.208654,0.000013
4128687,248772010,2022-01-02,3008152ce54bc95bc78b003206dc45a7,KPHL,Philadelphia,US,LSZH,Zurich,CH,2022-01-02T00:01:00Z,...,-58.601074,40975.0,543.000000,76.040526,-64.000000,248772010,49.665905,-31.158336,210.208643,0.000013
4128688,248772010,2022-01-02,3008152ce54bc95bc78b003206dc45a7,KPHL,Philadelphia,US,LSZH,Zurich,CH,2022-01-02T00:01:00Z,...,-58.601074,40975.0,543.000000,76.040526,-64.000000,248772010,49.665890,-31.158134,210.208632,0.000013


## Feature Encoding:

In [8]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [9]:
le.fit(df_combined['adep'])
df_combined['adep_category'] = le.transform(df_combined['adep']) + 1

In [10]:
le.fit(df_combined['ades'])
df_combined['ades_category'] = le.transform(df_combined['ades']) + 1

In [11]:
def encode_datetime(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

In [12]:
df_combined['actual_offblock_time'] = pd.to_datetime(df_combined['actual_offblock_time'], utc=True)
df_combined['actual_offblock_time_month'] = df_combined["actual_offblock_time"].dt.month
df_combined['actual_offblock_time_day'] = df_combined["actual_offblock_time"].dt.day
df_combined['actual_offblock_time_hour'] = df_combined["actual_offblock_time"].dt.hour
df_combined['actual_offblock_time_minue'] =df_combined["actual_offblock_time"].dt.minute
encode_datetime(df_combined, 'actual_offblock_time_month', 12)
encode_datetime(df_combined, 'actual_offblock_time_day', 31)
encode_datetime(df_combined, 'actual_offblock_time_hour', 24)
encode_datetime(df_combined, 'actual_offblock_time_minue', 60)

,flight_id,date,callsign,adep,name_adep,country_code_adep,ades,name_ades,country_code_ades,actual_offblock_time,...,actual_offblock_time_hour,actual_offblock_time_minue,actual_offblock_time_month_sin,actual_offblock_time_month_cos,actual_offblock_time_day_sin,actual_offblock_time_day_cos,actual_offblock_time_hour_sin,actual_offblock_time_hour_cos,actual_offblock_time_minue_sin,actual_offblock_time_minue_cos
0,248763780,2022-01-01,3840d84f25d3f5fcc0a1be3076bb4039,EGLL,London Heathrow,GB,EICK,Cork,IE,2022-01-01 13:46:00+00:00,...,13,46,0.5,0.866025,0.201299,0.979530,-0.258819,-0.965926,-0.994522,0.104528
1,248763780,2022-01-01,3840d84f25d3f5fcc0a1be3076bb4039,EGLL,London Heathrow,GB,EICK,Cork,IE,2022-01-01 13:46:00+00:00,...,13,46,0.5,0.866025,0.201299,0.979530,-0.258819,-0.965926,-0.994522,0.104528
2,248763780,2022-01-01,3840d84f25d3f5fcc0a1be3076bb4039,EGLL,London Heathrow,GB,EICK,Cork,IE,2022-01-01 13:46:00+00:00,...,13,46,0.5,0.866025,0.201299,0.979530,-0.258819,-0.965926,-0.994522,0.104528
3,248763780,2022-01-01,3840d84f25d3f5fcc0a1be3076bb4039,EGLL,London Heathrow,GB,EICK,Cork,IE,2022-01-01 13:46:00+00:00,...,13,46,0.5,0.866025,0.201299,0.979530,-0.258819,-0.965926,-0.994522,0.104528
4,248763780,2022-01-01,3840d84f25d3f5fcc0a1be3076bb4039,EGLL,London Heathrow,GB,EICK,Cork,IE,2022-01-01 13:46:00+00:00,...,13,46,0.5,0.866025,0.201299,0.979530,-0.258819,-0.965926,-0.994522,0.104528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4128685,248772010,2022-01-02,3008152ce54bc95bc78b003206dc45a7,KPHL,Philadelphia,US,LSZH,Zurich,CH,2022-01-02 00:01:00+00:00,...,0,1,0.5,0.866025,0.394356,0.918958,0.000000,1.000000,0.104528,0.994522
4128686,248772010,2022-01-02,3008152ce54bc95bc78b003206dc45a7,KPHL,Philadelphia,US,LSZH,Zurich,CH,2022-01-02 00:01:00+00:00,...,0,1,0.5,0.866025,0.394356,0.918958,0.000000,1.000000,0.104528,0.994522
4128687,248772010,2022-01-02,3008152ce54bc95bc78b003206dc45a7,KPHL,Philadelphia,US,LSZH,Zurich,CH,2022-01-02 00:01:00+00:00,...,0,1,0.5,0.866025,0.394356,0.918958,0.000000,1.000000,0.104528,0.994522
4128688,248772010,2022-01-02,3008152ce54bc95bc78b003206dc45a7,KPHL,Philadelphia,US,LSZH,Zurich,CH,2022-01-02 00:01:00+00:00,...,0,1,0.5,0.866025,0.394356,0.918958,0.000000,1.000000,0.104528,0.994522


In [13]:
df_combined['arrival_time'] = pd.to_datetime(df_combined['arrival_time'], utc=True)
df_combined['arrival_time_month'] = df_combined["arrival_time"].dt.month
df_combined['arrival_time_day'] = df_combined["arrival_time"].dt.day
df_combined['arrival_time_hour'] = df_combined["arrival_time"].dt.hour
df_combined['arrival_time_minue'] = df_combined["arrival_time"].dt.minute
encode_datetime(df_combined, 'arrival_time_month', 12)
encode_datetime(df_combined, 'arrival_time_day', 31)
encode_datetime(df_combined, 'arrival_time_hour', 12)
encode_datetime(df_combined, 'arrival_time_minue', 60)

,flight_id,date,callsign,adep,name_adep,country_code_adep,ades,name_ades,country_code_ades,actual_offblock_time,...,arrival_time_hour,arrival_time_minue,arrival_time_month_sin,arrival_time_month_cos,arrival_time_day_sin,arrival_time_day_cos,arrival_time_hour_sin,arrival_time_hour_cos,arrival_time_minue_sin,arrival_time_minue_cos
0,248763780,2022-01-01,3840d84f25d3f5fcc0a1be3076bb4039,EGLL,London Heathrow,GB,EICK,Cork,IE,2022-01-01 13:46:00+00:00,...,15,4,0.5,0.866025,0.201299,0.979530,1.0,1.194340e-15,0.406737,0.913545
1,248763780,2022-01-01,3840d84f25d3f5fcc0a1be3076bb4039,EGLL,London Heathrow,GB,EICK,Cork,IE,2022-01-01 13:46:00+00:00,...,15,4,0.5,0.866025,0.201299,0.979530,1.0,1.194340e-15,0.406737,0.913545
2,248763780,2022-01-01,3840d84f25d3f5fcc0a1be3076bb4039,EGLL,London Heathrow,GB,EICK,Cork,IE,2022-01-01 13:46:00+00:00,...,15,4,0.5,0.866025,0.201299,0.979530,1.0,1.194340e-15,0.406737,0.913545
3,248763780,2022-01-01,3840d84f25d3f5fcc0a1be3076bb4039,EGLL,London Heathrow,GB,EICK,Cork,IE,2022-01-01 13:46:00+00:00,...,15,4,0.5,0.866025,0.201299,0.979530,1.0,1.194340e-15,0.406737,0.913545
4,248763780,2022-01-01,3840d84f25d3f5fcc0a1be3076bb4039,EGLL,London Heathrow,GB,EICK,Cork,IE,2022-01-01 13:46:00+00:00,...,15,4,0.5,0.866025,0.201299,0.979530,1.0,1.194340e-15,0.406737,0.913545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4128685,248772010,2022-01-02,3008152ce54bc95bc78b003206dc45a7,KPHL,Philadelphia,US,LSZH,Zurich,CH,2022-01-02 00:01:00+00:00,...,7,14,0.5,0.866025,0.394356,0.918958,-0.5,-8.660254e-01,0.994522,0.104528
4128686,248772010,2022-01-02,3008152ce54bc95bc78b003206dc45a7,KPHL,Philadelphia,US,LSZH,Zurich,CH,2022-01-02 00:01:00+00:00,...,7,14,0.5,0.866025,0.394356,0.918958,-0.5,-8.660254e-01,0.994522,0.104528
4128687,248772010,2022-01-02,3008152ce54bc95bc78b003206dc45a7,KPHL,Philadelphia,US,LSZH,Zurich,CH,2022-01-02 00:01:00+00:00,...,7,14,0.5,0.866025,0.394356,0.918958,-0.5,-8.660254e-01,0.994522,0.104528
4128688,248772010,2022-01-02,3008152ce54bc95bc78b003206dc45a7,KPHL,Philadelphia,US,LSZH,Zurich,CH,2022-01-02 00:01:00+00:00,...,7,14,0.5,0.866025,0.394356,0.918958,-0.5,-8.660254e-01,0.994522,0.104528


In [14]:
le.fit(df_combined['aircraft_type'])
df_combined['aircraft_type_category'] = le.transform(df_combined['aircraft_type']) + 1

In [15]:
le.fit(df_combined['wtc'])
df_combined['wtc_category'] = le.transform(df_combined['wtc']) + 1

In [16]:
le.fit(df_combined['airline'])
df_combined['airline_category'] = le.transform(df_combined['airline']) + 1

In [17]:
df_combined.columns

Index(['flight_id', 'date', 'callsign', 'adep', 'name_adep',
       'country_code_adep', 'ades', 'name_ades', 'country_code_ades',
       'actual_offblock_time', 'arrival_time', 'aircraft_type', 'wtc',
       'airline', 'flight_duration', 'taxiout_time', 'flown_distance', 'tow',
       'timestamp', 'latitude', 'longitude', 'altitude', 'groundspeed',
       'track', 'vertical_rate', 'icao24', 'u_component_of_wind',
       'v_component_of_wind', 'temperature', 'specific_humidity',
       'adep_category', 'ades_category', 'actual_offblock_time_month',
       'actual_offblock_time_day', 'actual_offblock_time_hour',
       'actual_offblock_time_minue', 'actual_offblock_time_month_sin',
       'actual_offblock_time_month_cos', 'actual_offblock_time_day_sin',
       'actual_offblock_time_day_cos', 'actual_offblock_time_hour_sin',
       'actual_offblock_time_hour_cos', 'actual_offblock_time_minue_sin',
       'actual_offblock_time_minue_cos', 'arrival_time_month',
       'arrival_time_day', '

### Barometric Formula:

The air pressure $P$ at altitude $h$ is given by:

$$
P(h) = P_0 \cdot \exp\left( \frac{-Mgh}{RT} \right)
$$

Where:

- $P(h)$ is the pressure at altitude $h$,
- $P_0$ is the pressure at sea level (standard pressure: 101325 Pa),
- $M$ is the molar mass of Earth's air (approximately $0.029 \, \text{kg/mol}$),
- $g$ is the acceleration due to gravity $(9.81 \, \text{m/s}^2)$,
- $h$ is the altitude (in meters),
- $R$ is the universal gas constant $(8.314 \, \text{J/(mol·K)})$,
- $T$ is the temperature at altitude $h$ (in Kelvin).

### Air Density Formula:

The air density $rho$ is given by the ideal gas law:

$$
\rho = \frac{P}{R_s T}
$$

Where:
- $P$ is the pressure at altitude $h$,
- $R_s$ is the specific gas constant for dry air (approximately $287.05 \, \text{J/(kg·K)}$),
- $T$ is the temperature in Kelvin.

### Combined Formula for Air Density:

$$
\rho(h) = \frac{P_0 \cdot \exp\left( \frac{-Mgh}{RT} \right)}{R_s T}
$$

In [18]:
# Constants
p0 = 101325  # Pressure at sea level (Pa)
T0 = 288.15  # Standard temperature at sea level (K)
g = 9.80665  # Acceleration due to gravity (m/s²)
L = 0.0065  # Temperature lapse rate (K/m)
R = 8.3144598  # Universal gas constant (J/(mol·K))
M = 0.0289644  # Molar mass of Earth's air (kg/mol)
R_air = 287.05  # Specific gas constant for dry air (J/(kg·K))
C_L = 0.5  # Approximate lift coefficient (this can vary, but 0.5 is a reasonable starting point for climb)

In [19]:
# Function to calculate air density with humidity
def air_density_with_humidity(altitude, temperature, specific_humidity):
    temperature_kelvin = temperature + 273.15
    
    # Calculate pressure at altitude using barometric formula
    pressure = p0 * (1 - (L * altitude) / T0) ** (g * M / (R * L))
    
    # Calculate partial pressure of water vapor (e) using specific humidity
    e = (specific_humidity * pressure) / (0.622 + specific_humidity)
    
    # Calculate virtual temperature (T_v)
    virtual_temperature = temperature_kelvin * (1 + 0.61 * specific_humidity)
    
    # Calculate air density using the virtual temperature
    density = pressure / (R_air * virtual_temperature)
    
    return density    

## Wind speed
v = $\sqrt[2]{u\_wind^2+v\_wind^2}$

In [20]:
def wind_speed(u_wind, v_wind):
    return np.sqrt(u_wind **2 + v_wind ** 2)

## True Air speed
v = $\sqrt[2]{groundspeed^2+wind\_speed^2}$

In [21]:
# Function to calculate true airspeed with altitude-based wind components
def true_airspeed(groundspeed, u_wind, v_wind):
    return np.sqrt(groundspeed**2 + wind_Speed(u_wind, v_wind))

## estimate lift force

In [22]:
# df_trajectory['wind_speed'] = df_trajectory.apply(lambda row: wind_speed(row['u_component_of_wind'], row['v_component_of_wind']), axis=1)
df_combined['wind_speed'] = np.sqrt(df_combined['u_component_of_wind'] **2 + df_combined['v_component_of_wind'] ** 2)

In [23]:
df_combined['true_airspeed'] = np.sqrt(df_combined['groundspeed'] **2 + df_combined['wind_speed'] ** 2)

In [24]:
df_combined['pressure'] = p0 * (1 - (L * df_trajectory['altitude']) / T0) ** (g * M / (R * L))

In [25]:
# Calculate partial pressure of water vapor (e) using specific humidity
df_combined['pressure_e'] = (df_combined['specific_humidity'] * df_combined['pressure']) / (0.622 + df_combined['specific_humidity'])

In [26]:
# Calculate virtual temperature (T_v)
df_combined['virtual_temperature'] = (df_combined['temperature']+ 273.15) * (1 + 0.61 * df_combined['specific_humidity'])

In [27]:
df_combined['air_density'] = df_combined['pressure'] / (R_air * df_combined['virtual_temperature'])

In [28]:
# Assume K is some constant value based on typical aircraft performance
# (Note: This needs to be estimated or assumed. We'll use a placeholder value for now)
K = 0.5  # A reasonable placeholder value for K based on aircraft type

# Estimate lift force using L = K * rho * v^2
df_combined['lift_force'] = K * df_combined['air_density'] * df_combined['true_airspeed']**2

In [29]:
df_combined['lift_force']

0              0.602814
1              0.571135
2              0.540312
3              0.510344
4              0.481232
               ...     
4128685    32923.417820
4128686    32923.417020
4128687    33046.129461
4128688    33169.211991
4128689    33169.211185
Name: lift_force, Length: 4128690, dtype: float64

## Splitting Data

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [31]:
list(df_combined)

['flight_id',
 'date',
 'callsign',
 'adep',
 'name_adep',
 'country_code_adep',
 'ades',
 'name_ades',
 'country_code_ades',
 'actual_offblock_time',
 'arrival_time',
 'aircraft_type',
 'wtc',
 'airline',
 'flight_duration',
 'taxiout_time',
 'flown_distance',
 'tow',
 'timestamp',
 'latitude',
 'longitude',
 'altitude',
 'groundspeed',
 'track',
 'vertical_rate',
 'icao24',
 'u_component_of_wind',
 'v_component_of_wind',
 'temperature',
 'specific_humidity',
 'adep_category',
 'ades_category',
 'actual_offblock_time_month',
 'actual_offblock_time_day',
 'actual_offblock_time_hour',
 'actual_offblock_time_minue',
 'actual_offblock_time_month_sin',
 'actual_offblock_time_month_cos',
 'actual_offblock_time_day_sin',
 'actual_offblock_time_day_cos',
 'actual_offblock_time_hour_sin',
 'actual_offblock_time_hour_cos',
 'actual_offblock_time_minue_sin',
 'actual_offblock_time_minue_cos',
 'arrival_time_month',
 'arrival_time_day',
 'arrival_time_hour',
 'arrival_time_minue',
 'arrival_time_

In [32]:
X, y= df_combined[[
    'date',
    'adep',
    'country_code_adep',
    'ades',
    'country_code_ades',
    'arrival_time',
    'wtc',
    'airline',
    'flight_duration',
    'taxiout_time',
    'flown_distance', 
    'timestamp',
    'latitude',
    'longitude',
    'altitude',
    'track',
    'vertical_rate',
    'icao24',
    'temperature',
    'specific_humidity',
    'adep_category',
    'ades_category',
    'actual_offblock_time_month',
    'actual_offblock_time_day',
    'actual_offblock_time_hour',
    'actual_offblock_time_minue',
    'actual_offblock_time_month_sin',
    'actual_offblock_time_month_cos',
    'actual_offblock_time_day_sin',
    'actual_offblock_time_day_cos',
    'actual_offblock_time_hour_sin',
    'actual_offblock_time_hour_cos',
    'actual_offblock_time_minue_sin',
    'actual_offblock_time_minue_cos',
    'arrival_time_month',
    'arrival_time_day',
    'arrival_time_hour',
    'arrival_time_minue',
    'arrival_time_month_sin',
    'arrival_time_month_cos',
    'arrival_time_day_sin',
    'arrival_time_day_cos',
    'arrival_time_hour_sin',
    'arrival_time_hour_cos',
    'arrival_time_minue_sin',
    'arrival_time_minue_cos',
    'aircraft_type_category',
    'wtc_category',
    'airline_category',
    'wind_speed',
    'true_airspeed',
    'pressure',
    'pressure_e',
    'virtual_temperature',
    'air_density',
    'lift_force'
]], df_combined['tow']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()

In [ ]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Model

In [ ]:
from lightgbm import LGBMRegressor
from sklearn.metrics import root_mean_squared_error, r2_score


model = LGBMRegressor(learning_rate=0.5,force_row_wise=True)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

print("model score", model.score(X_train_scaled, y_train))
# 0.9863556751160256

print("rmse:", root_mean_squared_error(y_test, y_pred))
print("r2 score", r2_score(y_test, y_pred))